<img src="images/network_prof.png" width="150" alt="Prof looking at a electronic brain" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Dans cette partie, nous allons créer de vrais agents pour répondre aux questions des élèves sur le contenu du cours réseau. 

Avec le professeur de philosophie, nous avons vu les interrogrations de base d'un LLM, avec des appels de bas niveau.
Cela demandait pas mal de code, et surtout les interrogrations étaient séquentielles. Quand nous avons demander à plusieurs LLM de plancer sur le devoir, il a fallu attendre la réponse
de la première pour lancer la seconde, alors qu'elles auraient faire ce travail en même temps.

Voici la liste des modules dont nous aurons besoin dans cette partie

In [2]:
from dotenv import load_dotenv
import os
from pypdf import PdfReader
from IPython.display import Markdown, display 


from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput

import gradio
import asyncio
import requests 


## Retrouver le contenu du livre "Programmer l'Internet des Objets"

Dans un premier temps, nous allons convertir en texte les premières pages du livre.

In [3]:
book = PdfReader("./PLIDO_BOOK_en.pdf")
book_content = ""
for page in book.pages:
    text = page.extract_text()
    book_content += text

## Donner l'information au LLM

<img src="images/agent.png" width="150" alt="One agent" style="float: left; margin-right: 15px; margin-bottom: 10px;">Une fois les clés API chargées pour plusieurs serveurs (on utilisera par défaut celui de l'Université de Rennes, mais l'utilisation de Gemini est aussi possible). On donne l'URI et le Token pour le service, puis dans un deuxième temps, on précise le modèle LLM utilisé. Si on utilisait par defaut OpenAI, ces lignes seraient inutiles. Lors de l'appel d'```Agent``` , dans le cas d'OpenAI, le nom du modèle est directement indiqué dans une chaîne de caractères. 

A la création de l'Agent, lui donne un identifiant pour les traces, puis les instructions qui vont lui indiquer son rôle, les limites des réponses et les actions qu'il devra faire dans certain cas. Ces instructions contiennent également l'integralité du livre en ASCII. A noter que l'on demande au LLM de ne pas chercher à répondre en détail si la réponse ne se trouve pas dans le livre.

In [4]:
load_dotenv(override=True)

rennes_api_key = os.getenv("RENNES_API_KEY")
if not rennes_api_key:
    print("RENNES_API_KEY is missing")
    exit(1)

google_api_key = os.getenv("GOOGLE_API_KEY")
if not rennes_api_key:
    print("GOOLE_API_KEY is missing")
    exit(1)
    
RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

rennes_client = AsyncOpenAI(base_url=RENNES_BASE_URL, api_key=rennes_api_key)
rennes_model  = OpenAIChatCompletionsModel(model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", openai_client=rennes_client)

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model  = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

ollama_client = AsyncOpenAI(base_url=OLLAMA_BASE_URL, api_key="dont matter")
ollama_model  = OpenAIChatCompletionsModel(model="mistral:latest", openai_client=ollama_client)


instructions = f"""Voici le contenu d'un livre sur l'Internet des Objets

{book_content}

Ta responsabilité est de représenter l'auteur (Laurent Toutain) pour des interactions avec les élèves.
Les réponses doivent être professionnelles, claires, et doivent donner envie aux étudiants de s'engager
dans le cours, voire de choisir cette formation.
Si tu ne connais pas la réponse aux questions, répond Non."""

book_agent = Agent(name="PLIDO Book Agent", instructions=instructions, model=gemini_model)

Un agent se lance en utilisant la coroutine Python ```run``` du module ```Runner``` importé du module ```agents``` d'OpenAI.  L'utilisation du mot-clé ```await``` est indispensable. Ici, la différence avec une fonction est minime, vu que l'on ne lance qu'une coroutine et que l'on attend sa fin pour passer à l'instruction suivante. 

Vous pouvez relancer la cellule suivante plusieurs fois en changeant la question.

In [5]:
result = await Runner.run(book_agent, "Est ce que le livre est un bon livre à lire à la plage?")
display(Markdown(result.final_output))

En tant qu'assistant de Laurent Toutain, je dirais que cela dépend de votre définition d'une bonne lecture de plage ! 

Si vous cherchez une lecture légère et divertissante pour vous détendre complètement, ce livre est peut-être un peu dense pour la plage.

En revanche, si vous aimez utiliser votre temps à la plage pour apprendre et approfondir vos connaissances, et que le sujet de l'Internet des Objets vous passionne, alors ce livre peut être une lecture enrichissante, même à la plage. Imaginez-vous en train de décortiquer les protocoles IoT tout en profitant du soleil et du bruit des vagues !

Dans tous les cas, n'oubliez pas votre crème solaire et vos lunettes de soleil !


In [6]:
async def chat_async(message, history):
    """Fonction async pour l'agent"""
    try:
        result = await Runner.run(book_agent, message)
        return result.final_output
    except Exception as e:
        return f"Erreur: {e}"

def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


On peut voir que cela amène à quelques acrobaties dans Python. Gradio appelle une fonction en callback pour traiter la commande de l'utilisateur, et l'interaction avec l'agent se fait par une coroutine. D'où la fonction `chat_fn` qui ne fait qu'appeler la coroutine, attend la fin de son execution grâce à `asyncio.run` et retourne le résultat. 

# Plusieurs agents 

<img src="images/agents.png" width="150" alt="Several agents" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Nous allons reprendre une structure calssique pour voir comment exploiter le parallisme avec `asyncio`, nous allons demander à deux LLM de cogiter sur la même question et l'on prendra la meilleure des deux. Comme nous allons utiliser le modèle ollama en local, il y a peu de chance qu'elle fasse la réponse la plus futée, mais sait-on jamais.

In [7]:
ollama_agent = Agent(name="PLIDO Book Agent by Ollama", instructions=instructions, model=ollama_model)

instructions= """selectionne la réponse la plus claire parmi les différentes options à ces questions
d'étudiants sur un cours. Il faut prendre celui qui te deonnera le plus envie de suivre les cours.
Ne rajoute pas d'explication aux réponses possibles. Repond juste avec la mailleure réponse"""

best_answer  = Agent(name="Response selection", instructions=instructions, model=gemini_model)

async def chat_async(message, history):
    """Fonction async pour les appels au deux agents puis à la sélection"""
    
    results = await asyncio.gather(
        Runner.run(book_agent, message),
        Runner.run(ollama_agent, message),
    )
    outputs = [result.final_output for result in results]

    answers = "Réponses à la question:\n\n" + "\nRéponse:\n".join(outputs)
    best = await Runner.run(best_answer, answers)

    return best.final_output

def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


# Envoi de message
<img src="images/telephone.png" width="150" alt="Several agents" style="float: left; margin-right: 15px; margin-bottom: 10px;">
Nous pouvons demander au LLM d'interagir avec le monde extérieur. pour cela nous allons utiliser une application qui envoie des messages sur votre téléphone portable.

* téléchargez depuis votre magasin d'application (Android ou Apple) l'application `pushover`
* Créez votre compte
* Loguez vous également depuis votre ordinateur
* Recupérez votre clé d'utilisateur qui apparait en haut à droite et stockez là dans la la variable `PUSHOVER_USER_ID` dans le fichier `.env`
* En bas de la page, choisissez *Your Applications* et cliquez sur *Create an Application/Token*
  * Donnez un nom comme *PLIDOagent*, et une fois validé, un token va apparaître
  * Mettez ce token dans la variable `PUSHOVER_TOKEN` dans le fichier `.env`

Le petit programme ci dessus vous permet de teste si ca marche.

In [9]:
pushover_token=os.getenv("PUSHOVER_TOKEN")
pushover_user_id=os.getenv("PUSHOVER_USER_ID")
pushover_uri ="https://api.pushover.net/1/messages.json"

if not pushover_token or not pushover_user_id:
    print ("User_id or token missing")
    exit(1)

def push(message):
    payload = {"user": pushover_user_id, "token": pushover_token, "message": message}
    x = requests.post(pushover_uri, data=payload)

push("Hello my dear")


Nous allons décrire une fonction qui fait l'interface entre le push et le LLM. Pour cela nous allons utiliser le décorateur de fonctions `@function_tool` definit par openAI.

In [132]:
@function_tool
def send_message(object:str):
    """This function is used to send a message to the book author, to inform that you want to follow his class.
    If a student gives his name and wants to register uses this function to inform me.

    Args:
        - object: Protocol mane.`
    """

    message = f"Un etudiant s'interesse à  {object}."
    push(message)
    return {"status": "success"}

On peut donc modifier les instructions à notre agent et ajouter lors de sa création l'argument `tools`qui va contenir la liste des programmes qu'il peut appeler. Il va utiliser la description de la *doc string* au debut de la fonction pour comprendre comment l'utiliser.

In [18]:
instructions= """selectionne la réponse la plus claire parmi les différentes options à ces questions
d'étudiants sur un cours. Il faut prendre celui qui te deonnera le plus envie de suivre les cours.
Ne rajoute pas d'explication aux réponses possibles. Repond juste avec la mailleure réponse
Maisi tu detectes un nom de protocole dans les réponses envoie un message avec la fonction send_message et
le nom du protocol comme objet.
"""

best_answer  = Agent(name="Response selection", 
                     instructions=instructions, 
                     tools=[send_message],
                     model=gemini_model)

async def chat_async(message, history):
    """Fonction async pour les appels au deux agents puis à la sélection"""
    
    results = await asyncio.gather(
        Runner.run(book_agent, message),
        Runner.run(ollama_agent, message),
    )
    outputs = [result.final_output for result in results]

    answers = "Réponses à la question:\n\n" + "\nRéponse:\n".join(outputs)
    best = await Runner.run(best_answer, answers)

    return best.final_output

def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7892
* To create a public link, set `share=True` in `launch()`.


# De la vraie Agentique AI

Bon, là on peut interfacer des fonctions avec un LLM, donc l'étape d'après est de faire de même avec les Agents. Dans le code précédent, le cheminement est alogirthmique et guidé par le code Python que l'on a écrit:
* Deux Agents sont appelés pour fournir des réponses
* un troisième Agent analyse les réponses, choisi la meilleure et s'il détecte une référence à un protocole, envoie une alerte au professeur.

On va donc changer la logique du code, en transformant les deux Agents responsable des réponses en function et le troisième Agent va orchestrer l'ensemble du processus, c'est a dire appeler les fonctions (ex Agent) pour les réponse et s'il detecte un protocole, envoyer un message.

In [ ]:
tool_answer1 = book_agent.as_tool(tool_name="Book_agt_answer", tool_description="answer to user questions")
tool_answer2 = ollama_agent.as_tool(tool_name="Ollama_agt_answer", tool_description="answer to user questions")

tools = [tool_answer1, tool_answer2, send_message]

instructions ="""
Tu gères les réponses aux questions d'étudiant sur un livre et un cours réseau. Ton but est de selectionner la meilleure réponse
faite par des Agents qui ont la connaissance du contenu du cours et du livre. 

Si tu detecte dans la question ou la reponse que l'étudiant demande des informations concernant un protocole réseau précis, 
envoie un message à l'auteur du livre du cours  avec la fonction send_message et le nom du protocole, donne la réponse complète à 
l'étudiant et informe le que sa requete a été enregistrée a des fins de statistiques.

1: demande aux agents de répondre au question
2: selectionne la meilleure réponse dans le sens où la réponse contient des détails sur le cours et que le ton est courtois et engageant
3: si l'étudiant demande des informations concernant un protocole réseau précis, envoie un message au professeur responsable du cours 
avec la fonction send_message.

Règles importantes:
- tu dois imperativement utiliser les agents répondeurs pour faire les brouillons de réponse, ne le fait pas toi même.
- Retourne à l'étudiant, une seule des réponses proposées.
"""

global_agent = Agent("Global Agent", instructions=instructions, tools=tools, model=gemini_model)

async def chat_async(message, history):
    """Fonction async pour les appels au deux agents puis à la sélection"""
    
    result = await Runner.run(global_agent, message)

    return result.final_output
    
def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7908
* To create a public link, set `share=True` in `launch()`.
